In [9]:
import os 
from config.load_key import load_key
if not os.environ.get('OPENAI_API_KEY'):  # ✅ Correct
    os.environ['OPENAI_API_KEY']=load_key('openai_api_key')  # ✅ Correct
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate

# 语言交互

1. temperature: [0,2) 的参数，会根据这个参数决定模型生成的随机性。 当数字越大的时候，结果越随机；反之，结果越固定。
2. top_p: 这个决定每一个token是从多少候选token中选择。

In [2]:
llm=ChatOpenAI(model='gpt-4o-mini',temperature=0)
messages=[
    SystemMessage(content='You are a helpful assistant that can answer questions and help with tasks.'),
    HumanMessage(content='What is the capital of the moon?')
]
response=llm.invoke(messages)
print(response)

content='The Moon does not have a capital because it is not a sovereign entity or a country. The Moon is a natural satellite of Earth and does not have a government or political structure. However, various space agencies and organizations have conducted missions to the Moon, and there are plans for future exploration and potential habitation. If you have any other questions about the Moon or space exploration, feel free to ask!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 33, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CqyDO29cJsQFnOTMYbZ9WTtBjby1Z', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None

# 访问其他大模型
We could use langchain_community to request other models which is not supported by langchain itself:

In [ ]:
from langchain_community.chat_models import ChatTongyi

# 流式输出

In [7]:
stream=llm.stream(messages)
for chunk in stream:
    print(chunk.text(),end='')
    

/Users/tqa946816/anaconda3/envs/langchain-env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .text as a property instead (e.g., message.text).
  exec(code_obj, self.user_global_ns, self.user_ns)


The Moon does not have a capital because it is not a sovereign entity or a country. The Moon is a natural satellite of Earth and does not have a government or political structure. However, various space agencies and organizations have conducted missions to the Moon, and there are plans for future exploration and potential colonization, but no official capital exists.

# 提示词模版

In [11]:
prompt_template=ChatPromptTemplate.from_messages([
    ('system','You are a helpful assistant that can answer questions and help with tasks.'),
    ('user','{text}')
])
prompt=prompt_template.invoke({'text':'What is the capital of the moon'})
answer=llm.invoke(prompt)
print(answer)

content='The Moon does not have a capital because it is not a sovereign entity or a country. The Moon is a natural satellite of Earth and does not have a government or political structure. However, various space agencies and organizations have conducted missions to the Moon, and there are plans for future exploration and potential habitation. If you have any other questions about the Moon or space exploration, feel free to ask!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 32, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CqytU1X2nHggJm7kyzK2FH5lQEZ2C', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None

# LECL 
我们可以使用它组建一个复杂的聊天场景 （链条式服务）。

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableMap

parser=StrOutputParser()
chain=prompt_template|llm|parser
print(chain.invoke({'text':'What is the capital of the moon'}))

further_prompt = ChatPromptTemplate.from_messages([
    ('system', 'Translate the following text into Chinese.'),
    ('user', '{talk}')
])

further_chain=(RunnablePassthrough.assign(talk=chain)|further_prompt|llm|parser)
print(further_chain.invoke({'text':'What is the capital of the moon'}))







APIConnectionError: Connection error.

In [19]:
from langchain_core.runnables import RunnableLambda
prompt_template_zh=ChatPromptTemplate.from_messages([
    ('system','Translate the following text into Chinese.'),
    ('user','{text}')
])

prompt_template_fr=ChatPromptTemplate.from_messages([
    ('system','Translate the following text into French.'),
    ('user','{text}')
])

chain_zh=prompt_template_zh|llm|parser
chain_fr=prompt_template_fr|llm|parser
parallel_chain=RunnableMap({
    'zh':chain_zh,
    'fr':chain_fr
})

final_chain=parallel_chain |RunnableLambda(lambda x: x['zh']+'\n'+x['fr'])
print(final_chain.invoke({'text':'What is the capital of the moon'}))




月球的首都是什么？
Quelle est la capitale de la lune ?


# 保存聊天记录